# Enhancer - BC for in vitro saturation mutagenesis in pSA351 5' BC

In this notebook we will use the enhancer-barcode assignment sequencing data to assign each MPRA barcode to an enhancer. In R1 the enhancer barcode is sequenced, in R2 the MPRA barcode is sequenced. Importantly, with **MPRA barcode** we refer to the random barcode in the transcript (17bp), with **enhancer barcode** we refer to the barcode attached to the 5' of the enhancer sequence when designing the library (11 bp).

Create the directory to put the results and move to work there:

In [2]:
mkdir -p {path-to-dir}
cd {path-to-dir}

Load the modules that we need:

In [ ]:
module load FastQC/0.11.9-Java-11
module load cutadapt/4.2
module load SeqKit/2.4.0
module load fastp/0.23.2-GCC-10.3.0
module load Python/3.7.4-foss-2018a

Set the path to the sequencing data and scripts:

In [4]:
path_raw={path-to-data}
path_scripts={path-to-scripts}

## Step 1: Fastqc

In this step we will run fastqc to get some statistics on the sequencing quality. This will create html in the output folder, including quality per base pair, GC content and duplication levels.

In [ ]:
mkdir 01.fastqc
fastqc ${path_raw}MGE_* -o 01.fastqc

## Step 2: Removing adapter sequences and finding barcodes

Next we have to retrieve the enhancer barcode and the MPRA barcode from the reads. From R1 we will retrieve the enhancer barcode. For that, we need to identify the reads containing the 5' adapter (TCCCCAGTGCAAGTGCAG), trim it, and retrieve the first 11 bp, which correspoond to the enhancer barcode. The parameter we need to provide to cutadapt are:
* -g: The adapter sequence (in this case, TCCCCAGTGCAAGTGCAG)
* -j: The number of cores to use (e.g. 10)
* --discard-untrimmed: Discard reads that do not contain the adapter or do not pass filters
* -m: Minimal sequence length after removing the adapter (in this case, 11)
* -l: Keep the first X bp of the sequence (in this case, 11; the enhancer barcode sequence)
* -o: output directory

In [ ]:
mkdir 02.clean_reads
for file in ${path_raw}*_R1*.fastq.gz
do
    file_name=`basename ${file}`
    file_tag=${file_name%.fastq.gz}
    echo $file_tag
    cutadapt -g TCCCCAGTGCAAGTGCAG -j 10 --discard-untrimmed -m 11 -l 11  \
             -o 02.clean_reads/${file_tag}.clean.fastq.gz ${path_raw}${file_name}
done

Now we have to retrive the MPRA barcode from R2. In this case the adapter is CTGGCCCTCGCAGACA...GATCGGCGCGCCGGTCC and the barcode length is 17bp. We will also write the reverse complement sequence. The parameters for cutadapt here are:
* -g: The adapter sequence (in this case, CTGGCCCTCGCAGACA...GATCGGCGCGCCGGTCC)
* -j: The number of cores to use (e.g. 10)
* --discard-untrimmed: Discard reads that do not contain the adapter or do not pass filters
* -m: Minimal sequence length after removing the adapter (in this case, 17)
* -M: Maximum sequence length after removing the adapter (in this case, 17)
* -o: output directory

In [ ]:
for file in ${path_raw}*_R2*.fastq.gz
do
    file_name=`basename ${file}`
    file_tag=${file_name%.fastq.gz}
    echo $file_tag
    cutadapt -g CTGGCCCTCGCAGACA...GATCGGCGCGCCGGTCC -j 10 --discard-untrimmed -m 17 -M 17 \
        ${file} |\
        seqkit seq -r -p -o 02.clean_reads/${file_tag}.clean.fastq.gz
done

Finally, we will keep only sequences with quality above 30.

In [ ]:
for file in 02.clean_reads/*.clean.fastq.gz
do
echo ${file##*/}
fastp -e 30 --disable_length_filtering -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 \
      -o ${file%_*}_Q30.fastq.gz \
      -i $file
done

We can also show some stats from this files, for example here we are printing:
* The total number of reads
* The number of unique reads
* The percentage of unique reads
* The most common sequence
* The number of times the most common sequence occurs
* The frequency of the most common sequence

In [ ]:
module load mawk/1.3.4-20200120-foss-2018a
for i in 02.clean_reads/*_Q30.fastq.gz
do
file=${i#*/}
zcat $i | \
mawk -v 'OFS=\t' -v filename="${file%_combined*}" '
{
    # Only look at each sequence line
    if (NR%4==2) {
        read = $1
        # Add 1 to current read counter.
        counts[read] += 1
        # Count total number of reads seen so far.
        total_reads += 1
    }
}

END {
    for (read in counts) {
        if (counts[read] > max) {
            max = counts[read]
            max_readname = read
        }
    }
    unique_reads = length(counts)
    print filename, total_reads, unique_reads, (unique_reads * 100 / total_reads) "%", max_readname, max, (max * 100 / total_reads) "%"
}
' \
   | column -t
done

## Step 3: Assigning MPRA barcodes to enhancer barcodes

In [14]:
rm -rf 03.enhancer_bc_assignment
mkdir 03.enhancer_bc_assignment

First we will reformat the enhancer barcodes fastq file to a table containing Read ID and enhancer barcode.

In [ ]:
module load mawk/1.3.4-20200120-foss-2018a
for file in 02.clean_reads/*_R1_Q30.fastq.gz
do
echo ${file}
file_name=`basename ${file}`
file_tag=${file_name%_R1_Q30.fastq.gz}
paste \
   <( zcat ${file} | mawk 'NR%4==1{print substr($1,2)}' ) \
   <( zcat ${file} | mawk 'NR%4==2' ) \
   | sort -k1 \
   | gzip -c \
   > 03.enhancer_bc_assignment/${file_tag}_readID_enhancerBarcode.tsv.gz
done

Similarly, we will reformat the MPRA barcodes fastq file to a table containing Read ID and CHEQ-seq barcode.

In [ ]:
for file in 02.clean_reads/*_R2_Q30.fastq.gz
do
file_name=`basename ${file}`
file_tag=${file_name%_R2_Q30.fastq.gz}
echo ${file_tag}
paste \
   <( zcat ${file} | mawk 'NR%4==1{print substr($1,2)}' ) \
   <( zcat ${file} | mawk 'NR%4==2' ) \
   | sort -k1 \
   | gzip -c \
   > 03.enhancer_bc_assignment/${file_tag}_readID_CHEQseqBarcode.tsv.gz
done

Now we will combine these two tables based on Read ID to obtain a table with Read ID, enhancer barcode and MPRA barcode.

In [12]:
for file in 02.clean_reads/*_R2_Q30.fastq.gz
do
file_name=`basename ${file}`
file_tag=${file_name%_R2_Q30.fastq.gz}
awk -F '\t' -v OFS='\t' 'FNR==NR{a[$1]=$2 FS $3;next}{ print $0, a[$1]}' \
   <( zcat 03.enhancer_bc_assignment/${file_tag}_readID_enhancerBarcode.tsv.gz ) \
   <( zcat 03.enhancer_bc_assignment/${file_tag}_readID_CHEQseqBarcode.tsv.gz ) \
   | awk  '$3!=""' \
   | cut -f1-3 \
   | gzip -c \
   > 03.enhancer_bc_assignment/${file_tag}_readID_enhancerBC_CHEQseqBC_coupled.tsv.gz
done

We will link the MPRA enhancer with the original enhancer ID. We have a python script which will link both files based on the enhancer barcode and return a enhancerID-CHEQ-seqBC table (without unassigned MPRA barcodes and MPRA barcodes that are linked to more than one barcode).

In [ ]:
module load Python/3.7.4-GCCcore-6.4.0
for file in 03.enhancer_bc_assignment/*_readID_enhancerBC_CHEQseqBC_coupled.tsv.gz
do
file_name=`basename ${file}`
file_tag=${file_name%_readID_enhancerBC_CHEQseqBC_coupled.tsv.gz}
echo ${file_tag}
${path_scripts}link_cheqseq_bc_to_enhancer_id.py -ebc IVSM_enhancer_to_bc.tsv -rbc ${file} \
                                                 -o 03.enhancer_bc_assignment/${file_tag}_enhancerID_CHEQseqBarcode_coupled.tsv.gz -rthr 10 -cthr 0.95
done

Create table with BC count per enhancer

In [ ]:
for file in 03.enhancer_bc_assignment/Recovered_MGE__*_enhancerID_CHEQseqBarcode_coupled.tsv.gz
do
echo ${file#/*}
zcat $file | awk '{print $1}' | sort | uniq -c > ${file%*.tsv.gz}BC-count.tsv
done

Now we can check how many enhancers have been found in the collection (the maximum number is 1288).

In [ ]:
for i in 03.enhancer_bc_assignment/*_enhancerID_CHEQseqBarcode_coupled.tsv.gz
do
echo ${i##*/}
zcat $i | cut -f1 | sort -u | wc -l
done

In [ ]:
# Number of enhancers with more than 5 or 10 BC
awk 'NR==1 { print } NR != 1 && $1 >= 5 { print }' 03.enhancer_bc_assignment/Recovered_MGE__ba667c__pSA351_lenti5_BC_microglia_enh_BC_enhancerID_CHEQseqBarcode_coupledBC-count.tsv | wc -l
awk 'NR==1 { print } NR != 1 && $1 >= 10 { print }' 03.enhancer_bc_assignment/Recovered_MGE__ba667c__pSA351_lenti5_BC_microglia_enh_BC_enhancerID_CHEQseqBarcode_coupledBC-count.tsv | wc -l

In [ ]:
for file in 03.enhancer_bc_assignment/Recovered_MGE__*_enhancerID_CHEQseqBarcode_coupled.tsv.gz
do
middleBC=$(($(zcat $file | awk '{print $1}'| sort | uniq -c | wc -l)/2))
echo ${file#/*}
echo "Number of BC associated to an enhancer: " $(zcat $file | wc -l)
echo "Enhancer coverage: " $(zcat $file | awk '{print $1}' | sort -u | wc -l)
echo "Number of enhancers with >= 5 BC" $(zcat $file | awk '{print $1}' | sort | uniq -c | awk 'NR==1 { print } NR != 1 && $1 >= 5 { print }' | wc -l)
echo "Median number of BC per enhancer: " $(zcat $file | awk '{print $1}' | sort | uniq -c | awk '{print $1}' | sort -g | sed -n ${middleBC}p)
echo "Number of shuffled regions: " $(zgrep 'Shuffle' $file | awk '{print $1}' | sort -u | wc -l)
done

## Step 4: Sequencing saturation

The next step is to check the sequencing saturation. For this we compare the number of reads versus the total number of reads, if they are proportional (a straight line) it indicates that we need more sequencing; if it reaches a plateau (the number of unique reads does not increase when increasing the number of reads), then the sample is sufficiently sequenced. First we prepare the files.

In [ ]:
# Extract DNA sequences from fastq
mkdir 04.sequencing_saturation
for file in 02.clean_reads/*_Q30.fastq.gz
do
file_name=`basename ${file}`
file_tag=${file_name%.fastq.gz}
echo $file_name
zcat $file | sed -n '2~4p' > 04.sequencing_saturation/${file_tag}.txt
done

In [ ]:
export LC_ALL=C
for file in 04.sequencing_saturation/MGE__*_Q30.txt
do
file_name=`basename ${file}`
file_tag=${file_name%.txt}
echo ${file_name}
nreads=$(cat $file | wc -l)
if test $nreads -gt 10000000
then
        increment=$(($nreads / 50))
else
        increment=200000
fi
    for i in $(seq 0 $increment $nreads)
    do
     # Print number of total and unique reads for every subsamples
    echo "echo $i \$(shuf -n $(($i)) $file | sort -u | wc -l) >> 04.sequencing_saturation/${file_tag}_sat.txt" \
    >> 04.sequencing_saturation/${file_tag}_parallel.txt
    done
cat 04.sequencing_saturation/${file_tag}_parallel.txt | parallel -j 16
 # Put values in numerical order
cat  04.sequencing_saturation/${file_tag}_sat.txt | sort -g > 04.sequencing_saturation/${file_tag}_satsort.txt && mv 04.sequencing_saturation/${file_tag}_satsort.txt 04.sequencing_saturation/${file_tag}_sat.txt
done

To create the plots we can use a python script.

In [5]:
module load Python/3.7.4-GCCcore-6.4.0
${path_scripts}plot_saturation.py -ssd 04.sequencing_saturation -r _sat.txt --ncol 2 --nrow 2